In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase import firebase
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib
import urllib.request as ur
import sys

2023-05-10 21:00:57.491509: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 21:00:57.524802: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 21:00:57.659121: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 21:00:57.660535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 21:00:58.702104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [8]:
model = tf.keras.models.load_model('/home/vh/Documents/Plant-Disease-Website/model/saved_model', compile=False)

2023-05-10 21:02:41.754187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
cred = credentials.Certificate('/home/vh/Documents/serviceAccount.json')

app = firebase_admin.initialize_app(cred)

In [12]:
db = firestore.client()

images_ref = db.collection(u'images')

query = images_ref.where(u'status', u'==', u'')

docs = query.stream()

urls = []

for doc in docs:
    doc_dict = doc.to_dict()

    imageUrl = doc_dict['imageUrl']
    
    urls.append(imageUrl)

print("Num of images: ",len(urls))

/home/vh/.local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Num of images:  1


In [15]:
images = []
for i in range(0,len(urls)):
    image = "image_" + str(i) +".jpg"
    images.append(image)
    urllib.request.urlretrieve(urls[i], image)
    with PIL.Image.open(image) as img:
        img = img.resize((256,256))
        img.save(f'./{image}')

In [16]:
print(images)
test = pd.DataFrame({'image':images})
print(test)

['image_0.jpg']
         image
0  image_0.jpg


In [17]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )
test_data = datagen.flow_from_dataframe(
    test,
    directory = '.',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

print(test_data)

preds = model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

Found 1 validated image filenames.


2023-05-10 21:04:03.821388: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 2s 2s/step
[[0.07096343 0.05563545 0.0110971  0.17978364 0.00778604 0.85357535]]
[[5]]


In [18]:
labels = {'complex': 0, 'frog_eye_leaf_spot': 1, 'healthy': 2, 'powdery_mildew': 3, 'rust': 4, 'scab': 5}
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []
temp = []
for i in indices:
    i = [i[0]]
    temp.append(i)
indices = temp

for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

{0: 'complex', 1: 'frog_eye_leaf_spot', 2: 'healthy', 3: 'powdery_mildew', 4: 'rust', 5: 'scab'}
['scab']


In [19]:
# cred = credentials.Certificate('/home/vh/Documents/serviceAccount.json')

# firebase_admin.initialize_app(cred)

# Initialize Firestore client

db = firestore.client()

firebase = firebase.FirebaseApplication('https://plant-pests-detection-default-rtdb.firebaseio.com/', None)

images_ref = db.collection(u'images')

# Create a query to filter documents by status equal to an empty string
query = images_ref.where(u'status', u'==', u'')

# Get the documents that match the query
docs = query.stream()

index = 0

for doc in docs:
    result = testlabels[index]
    
    detect_ref = db.collection(u'images').document(doc.id)

    detect_ref.update({u'status': result})

    firebase.put('status', 'status', result)
    
    index += 1

/home/vh/.local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
